In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
col_order = ['Batch', 'Plate_Map_Name','Assay_Plate_Barcode', 'Perturbation', 'Cell_type', 'Time', 'Density', 'Antibiotics', 'Cell_line', 'Time_delay', 'Times_imaged', 'Anomaly','Number_of_images']

antibiotics_df = pd.DataFrame({'Assay_Plate_Barcode': ['BR00118049','BR00117002', 'BR00117001'], 'Antibiotics': ['Blasticidin', 'Puromycin', 'Puromycin']})

density_df = pd.DataFrame({'Assay_Plate_Barcode': ['BR00117054', 'BR00117055', 'BR00117008', 'BR00117009'], 'Density': ['120', '120', '80', '80']})

cell_line_df = pd.DataFrame({'Assay_Plate_Barcode': ['BR00117052', 'BR00117053', 'BR00117050', 'BR00117051'], 'Cell_line': ['Cas9', 'Cas9', 'Cas9', 'Cas9']})

anomalous_plates = {
    'Phalloidin': ['BR00118050',
                   'BR00117006',
                   'BR00118049',
                   'BR00118039',
                   'BR00118040',
                   'BR00118041',
                   'BR00118042',
                   'BR00118043',
                   'BR00118044',
                   'BR00118045',
                   'BR00118046',
                   'BR00118047',
                   'BR00118048'],
    'WGA': ['BR00116997',
            'BR00116998',
            'BR00116999',
            'BR00116996',
            'BR00116991',
            'BR00116992',
            'BR00116993',
            'BR00116994',
            'BR00116995',
            'BR00117024',
            'BR00117025',
            'BR00117026'],
    'Mitotracker': ['BR00117013',
                    'BR00117010']
}

In [3]:
# Read barcode_platemap files
path = os.path.join('../metadata/platemaps/*/barcode_platemap.csv')
files = glob.glob(path)
df = pd.concat(
    (
        (
            pd.read_csv(_)
            .assign(Batch=_.split('/')[3])
            .assign(Anomaly='none')
            .assign(Time_delay='Day0')
            .assign(Times_imaged='1')
            .assign(Time='0')
        ) for _ in files),
    ignore_index=True
)

# Read number of images csv
path = os.path.join('input/2020*.csv')
files = glob.glob(path)
images_df = pd.concat((pd.read_csv(_) for _ in files), ignore_index=True)

df['Perturbation'] = np.asarray(list(df.Plate_Map_Name.str.split('_').values))[:,1]

In [4]:
# Add antibiotics column
df = df.merge(antibiotics_df, on='Assay_Plate_Barcode', how='left')
df.Antibiotics.replace(np.nan, 'absent', inplace=True)
df.query('Antibiotics!="absent"').head()

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics
49,BR00118049,JUMP-Target-1_orf_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,orf,Blasticidin
69,BR00117002,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,Puromycin
70,BR00117001,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,Puromycin


In [5]:
# Add density column
df = df.merge(density_df, on='Assay_Plate_Barcode', how='left')
df.Density.replace(np.nan, '100', inplace=True)
df.query('Density!="100"').head()

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density
91,BR00117054,JUMP-Target-1_compound_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,compound,absent,120
92,BR00117055,JUMP-Target-1_compound_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,compound,absent,120
93,BR00117008,JUMP-Target-1_compound_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,compound,absent,80
94,BR00117009,JUMP-Target-1_compound_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,compound,absent,80


In [6]:
# Add Cell line column
df = df.merge(cell_line_df, on='Assay_Plate_Barcode', how='left')
df.loc[df.Perturbation=='crispr','Cell_line'] = 'Cas9'
df.Cell_line.replace(np.nan, 'Parental', inplace=True)
df.query('Cell_line=="Cas9"').head(50)

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line
57,BR00118041,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
58,BR00118042,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
59,BR00118043,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
60,BR00118044,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
61,BR00118045,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
62,BR00118046,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
63,BR00118047,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
64,BR00118048,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
65,BR00117003,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9
66,BR00117004,JUMP-Target-1_crispr_platemap,2020_11_04_CPJUMP1,none,Day0,1,0,crispr,absent,100,Cas9


In [7]:
# Fill the Time delay column
df.loc[df.Batch.str.contains('Day1'), 'Time_delay'] = 'Day1'
df.loc[df.Batch.str.contains('Day4'), 'Time_delay'] = 'Day4'
df.loc[df.Batch.str.contains('2Weeks'), 'Time_delay'] = 'Week2'
df.loc[df.Batch.str.contains('4Weeks'), 'Time_delay'] = 'Week4'
df.query('Time_delay!="Day0"').head(50)

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line
24,BR00117006,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
25,BR00118050,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
26,BR00118039,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
27,BR00118040,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
28,BR00117020,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
29,BR00117021,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
30,BR00117022,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
31,BR00117023,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,none,Week2,1,0,orf,absent,100,Parental
32,BR00117006,JUMP-Target-1_orf_platemap,2020_11_19_TimepointDay4,none,Day4,1,0,orf,absent,100,Parental
33,BR00118050,JUMP-Target-1_orf_platemap,2020_11_19_TimepointDay4,none,Day4,1,0,orf,absent,100,Parental


In [8]:
# Fill the Times imaged column
df.loc[df.Assay_Plate_Barcode.str.endswith('A'), 'Times_imaged'] = '2'
df.loc[df.Assay_Plate_Barcode.str.endswith('B'), 'Times_imaged'] = '3'
df.loc[df.Assay_Plate_Barcode.str.endswith('C'), 'Times_imaged'] = '4'
df.loc[df.Assay_Plate_Barcode.str.endswith('D'), 'Times_imaged'] = '5'
df.loc[df.Assay_Plate_Barcode.str.endswith('E'), 'Times_imaged'] = '6'
df.loc[df.Assay_Plate_Barcode.str.endswith('F'), 'Times_imaged'] = '7'

df.loc[df.Time_delay=="Week2", 'Times_imaged'] = '2'
df.loc[df.Time_delay=="Week4", 'Times_imaged'] = '3'

df.query('Times_imaged=="3"').head(50)

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line
1,BR00116991B,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,3,0,compound,absent,100,Parental
7,BR00116992B,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,3,0,compound,absent,100,Parental
13,BR00116993B,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,3,0,compound,absent,100,Parental
19,BR00116994B,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,3,0,compound,absent,100,Parental
40,BR00117006,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,none,Week4,3,0,orf,absent,100,Parental
41,BR00118050,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,none,Week4,3,0,orf,absent,100,Parental
42,BR00118039,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,none,Week4,3,0,orf,absent,100,Parental
43,BR00118040,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,none,Week4,3,0,orf,absent,100,Parental
44,BR00117020,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,none,Week4,3,0,orf,absent,100,Parental
45,BR00117021,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,none,Week4,3,0,orf,absent,100,Parental


In [9]:
# Fill anomalies column
for anomaly in anomalous_plates:
    for plate in anomalous_plates[anomaly]:
        df.loc[df.Assay_Plate_Barcode==plate, 'Anomaly'] = anomaly
df.query('Anomaly!="none"').head(50)

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line
24,BR00117006,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,Phalloidin,Week2,2,0,orf,absent,100,Parental
25,BR00118050,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,Phalloidin,Week2,2,0,orf,absent,100,Parental
26,BR00118039,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,Phalloidin,Week2,2,0,orf,absent,100,Parental
27,BR00118040,JUMP-Target-1_orf_platemap,2020_12_02_CPJUMP1_2WeeksTimePoint,Phalloidin,Week2,2,0,orf,absent,100,Parental
32,BR00117006,JUMP-Target-1_orf_platemap,2020_11_19_TimepointDay4,Phalloidin,Day4,1,0,orf,absent,100,Parental
33,BR00118050,JUMP-Target-1_orf_platemap,2020_11_19_TimepointDay4,Phalloidin,Day4,1,0,orf,absent,100,Parental
34,BR00118039,JUMP-Target-1_orf_platemap,2020_11_19_TimepointDay4,Phalloidin,Day4,1,0,orf,absent,100,Parental
35,BR00118040,JUMP-Target-1_orf_platemap,2020_11_19_TimepointDay4,Phalloidin,Day4,1,0,orf,absent,100,Parental
40,BR00117006,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,Phalloidin,Week4,3,0,orf,absent,100,Parental
41,BR00118050,JUMP-Target-1_orf_platemap,2020_12_07_CPJUMP1_4WeeksTimePoint,Phalloidin,Week4,3,0,orf,absent,100,Parental


In [10]:
# Add Number of images column
df = df.merge(images_df, on=['Batch','Assay_Plate_Barcode'], how='left')
df.head()

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line,Description,Number_of_images
0,BR00116991A,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,2,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504
1,BR00116991B,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,3,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504
2,BR00116991C,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,4,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504
3,BR00116991D,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,5,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504
4,BR00116991E,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,6,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504


In [11]:
# Create Cell type column
df['Cell_type'] = list(df.Description.str[0:4])
df.head()

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line,Description,Number_of_images,Cell_type
0,BR00116991A,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,2,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504,A549
1,BR00116991B,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,3,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504,A549
2,BR00116991C,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,4,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504,A549
3,BR00116991D,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,5,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504,A549
4,BR00116991E,JUMP-Target-1_compound_platemap,2020_12_08_CPJUMP1_Bleaching,none,Day0,6,0,compound,absent,100,Parental,A549 24-hour Compound Plate 1,21504,A549


In [12]:
# Create Time column
df.loc[df.Description.str.contains('24'), 'Time'] = '24'
df.loc[df.Description.str.contains('48'), 'Time'] = '48'
df.loc[df.Description.str.contains('96'), 'Time'] = '96'
df.loc[df.Description.str.contains('144'), 'Time'] = '144'
df.loc[df.Time=='0']

,Assay_Plate_Barcode,Plate_Map_Name,Batch,Anomaly,Time_delay,Times_imaged,Time,Perturbation,Antibiotics,Density,Cell_line,Description,Number_of_images,Cell_type


In [13]:
# Sort rows and columns
df = df.sort_values(by=['Batch','Plate_Map_Name','Assay_Plate_Barcode'], ignore_index=True)[col_order]

In [14]:
df.to_csv('output/experiment-metadata.tsv', index=False, sep='\t')